# PyCity Schools Analysis
<ul>
    <li>The passing rate is much lower for District schools compared to Charter schools.</li>
    <li>Interestingly, Spending per Student/School does not appear to have a direct correlation on passing rates. Some schools who spend more than others tend to have lower passing rates, but other schools that spend the same amount often have higher passing rates. </li>
    <li>Unsuprisingly, Math tends to have the lower scores across all schools.</li>
    <li>District schools have much higher Total Student counts than Charter schools. Because there is such a big difference, it makes it hard to concretely concluded specific insights when comparing school types. Schools with more students tend to have more budget, but that does not strictly mean that a greater budget results in higher passing rates as explained before.</li>
    <li>Math/Reading scores by Grades do not show any meaningful differences between grades. This indicates that there is not a specific grade that is having issues, it is the entire school.</li>
    <li>Larger schools have a much lower overall passing rate, this is likely due to a number of issues beyond this dataset such as income, but we can assume that students in larger schools may get less one-on-one attention which leads to lower scores.</li>
</ul>
<hr>

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


<h2>District Summary</h2>

In [2]:
school_count = school_data_complete["school_name"].nunique()

School Count: 15


In [3]:
student_count = school_data_complete["student_name"].count()

Student Count: 39170


In [4]:
unique_schools = school_data_complete.drop_duplicates(subset=['school_name'])
total_budget = unique_schools["budget"].sum()

Total Budget: $24649428


In [5]:
average_math_score = school_data_complete["math_score"].mean()

Average Math Score: 78.98537145774827


In [6]:
average_reading_score = school_data_complete["reading_score"].mean()

Average Reading Score: 81.87784018381414


In [7]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100

Passing Math Percentage: 74.9808526933878%


In [8]:
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100

In [9]:
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

Overall Passing Rate: 65.17232575950983%


In [10]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({
    "Total Schools": [school_count],
    "Total Students": [student_count],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Average Reading Score": [average_reading_score],
    "% Passing Math": [passing_math_percentage],
    "% Passing Reading": [passing_reading_percentage],
    "% Overall Passing": [overall_passing_rate]
})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


<h2>School Summary</h2>

In [11]:
school_types = school_data.set_index(["school_name"])["type"]
school_types
school_types_df = school_types.reset_index(name='School Type')

In [12]:
per_school_counts = school_data_complete.groupby("school_name").size()
per_school_counts
per_school_counts_df = per_school_counts.reset_index(name='Student Count')

In [13]:
# School budgets
per_school_budget = school_data.set_index(["school_name"])["budget"]
per_school_budget
per_school_budget_df = per_school_budget.reset_index(name='Total School Budget')

In [14]:
# School budget per capita
school_data['per_capita_spending'] = school_data['budget'] / school_data['size']
budget_per_capita = school_data.set_index(["school_name"])["per_capita_spending"]
budget_per_capita_df = budget_per_capita.reset_index(name='Per Student Budget')

In [15]:
# Average MATH test scores per school
per_school_math = school_data_complete.groupby("school_name").agg({"math_score": "mean"}).reset_index()

In [16]:
# Average READING test scores per school
per_school_reading = school_data_complete.groupby("school_name").agg({"reading_score": "mean"}).reset_index()

In [17]:
# Students Passing Math Per School
students_passing_math = school_data_complete[school_data_complete["math_score"] >= 70]
school_students_passing_math = students_passing_math.groupby("school_name").size()
school_students_passing_math_df = school_students_passing_math.reset_index(name='Students Passing Math')

In [18]:
# Students Passing Reading Per School
students_passing_reading= school_data_complete[school_data_complete["reading_score"] >= 70]
school_students_passing_reading = students_passing_reading.groupby("school_name").size()
school_students_passing_reading_df = school_students_passing_reading.reset_index(name='Students Passing Reading')

In [19]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher

students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()
school_students_passing_math_and_reading_df = school_students_passing_math_and_reading.reset_index(name='Students Passing Math and Reading')

In [20]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_students_passing_math / per_school_counts * 100
per_school_passing_reading = school_students_passing_reading / per_school_counts * 100
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100
overall_passing_rate_df = overall_passing_rate.reset_index(name='Overall Passing Rate')

In [21]:
per_school_passing_math.head()
per_school_passing_math_df = per_school_passing_math.reset_index(name='Passing Math Rate')

In [22]:
per_school_passing_reading.head()
per_school_passing_reading_df = per_school_passing_reading.reset_index(name='Passing Reading Rate')

In [23]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
from functools import reduce

dataframes = [school_types_df, per_school_counts_df, per_school_budget_df, budget_per_capita_df, per_school_math, per_school_reading, school_students_passing_math_df, school_students_passing_reading_df, school_students_passing_math_and_reading_df, per_school_passing_math_df, per_school_passing_reading_df, overall_passing_rate_df]

per_school_summary = reduce(lambda left, right: pd.merge(left, right, on='school_name'), dataframes)

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Renaming Columns
per_school_summary = per_school_summary.rename(columns={"school_name":"School Name",
                                                        "math_score":"Average Math Score",
                                                        "reading_score":"Average Reading Score",
                                                        "Passing Math Rate":"% Passing Math",
                                                        "Passing Reading Rate":"% Passing Reading",
                                                        "Overall Passing Rate":"% Overall Passing",
                                                        "Student Count": "Total Students"})
# Setting Index
per_school_summary.set_index('School Name', inplace=True)

# Display the DataFrame
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Students Passing Math and Reading,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,,,,
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,1916,2372,1561,65.683922,81.316421,53.513884
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,1946,2381,1569,65.988471,80.739234,53.204476
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,1653,1688,1583,93.867121,95.854628,89.892107
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,3094,3748,2481,66.752967,80.862999,53.527508
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,1371,1426,1330,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,2143,2204,2068,93.867718,96.539641,90.582567
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,1749,1803,1697,94.133477,97.039828,91.334769
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,3318,4077,2719,66.680064,81.933280,54.642283
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,395,411,381,92.505855,96.252927,89.227166


<h2>Highest-Performing Schools (by % Overall Passing)</h2>

In [25]:
top_schools = per_school_summary.sort_values("% Overall Passing", ascending=False)
top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Students Passing Math and Reading,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,1749,1803,1697,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,1525,1591,1487,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,1371,1426,1330,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,2143,2204,2068,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,910,923,871,94.594595,95.945946,90.540541


<h2>Bottom Performing Schools (By % Overall Passing)</h2>

In [26]:
bottom_schools = per_school_summary.sort_values("% Overall Passing", ascending=True)
bottom_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Students Passing Math and Reading,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,2654,3208,2119,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,1946,2381,1569,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,1916,2372,1561,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,3094,3748,2481,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,3145,3867,2549,66.057551,81.222432,53.539172


<h2>Math Scores by Grade</h2>

In [27]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = ninth_graders.groupby('school_name')['math_score'].mean()
tenth_grader_math_scores = tenth_graders.groupby('school_name')['math_score'].mean()
eleventh_grader_math_scores = eleventh_graders.groupby('school_name')['math_score'].mean()
twelfth_grader_math_scores = twelfth_graders.groupby('school_name')['math_score'].mean()

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`

math_scores_by_grade = pd.DataFrame({
    "9th": ninth_grade_math_scores,
    "10th": tenth_grader_math_scores,
    "11th": eleventh_grader_math_scores,
    "12th": twelfth_grader_math_scores
})


# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade


,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


<h2>Reading Scores By Grade</h2>

In [28]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `reading_score` column for each.
ninth_grade_reading_scores = ninth_graders.groupby('school_name')['reading_score'].mean()
tenth_grader_reading_scores = tenth_graders.groupby('school_name')['reading_score'].mean()
eleventh_grader_reading_scores = eleventh_graders.groupby('school_name')['reading_score'].mean()
twelfth_grader_reading_scores = twelfth_graders.groupby('school_name')['reading_score'].mean()

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`

reading_scores_by_grade = pd.DataFrame({
    "9th": ninth_grade_reading_scores,
    "10th": tenth_grader_reading_scores,
    "11th": eleventh_grader_reading_scores,
    "12th": twelfth_grader_reading_scores
})


# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade


,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores By School Spending

In [29]:
# Establish the bins
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [30]:
# Create a copy of the school summary since it has the "Per Student Budget"
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].str.replace('$', '').str.replace(',', '').astype(float)
per_school_summary["Per Student Budget"] = pd.to_numeric(per_school_summary["Per Student Budget"], errors='coerce')
school_spending_df = per_school_summary.copy()

In [31]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"],
                                                             spending_bins, labels=labels,
                                                             include_lowest=True)

school_spending_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Students Passing Math and Reading,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
School Name,,,,,,,,,,,,,
Huang High School,District,2917,"$1,910,635.00",655.0,76.629414,81.182722,1916,2372,1561,65.683922,81.316421,53.513884,$645-680
Figueroa High School,District,2949,"$1,884,411.00",639.0,76.711767,81.158020,1946,2381,1569,65.988471,80.739234,53.204476,$630-645
Shelton High School,Charter,1761,"$1,056,600.00",600.0,83.359455,83.725724,1653,1688,1583,93.867121,95.854628,89.892107,$585-630
Hernandez High School,District,4635,"$3,022,020.00",652.0,77.289752,80.934412,3094,3748,2481,66.752967,80.862999,53.527508,$645-680
Griffin High School,Charter,1468,"$917,500.00",625.0,83.351499,83.816757,1371,1426,1330,93.392371,97.138965,90.599455,$585-630
Wilson High School,Charter,2283,"$1,319,574.00",578.0,83.274201,83.989488,2143,2204,2068,93.867718,96.539641,90.582567,<$585
Cabrera High School,Charter,1858,"$1,081,356.00",582.0,83.061895,83.975780,1749,1803,1697,94.133477,97.039828,91.334769,<$585
Bailey High School,District,4976,"$3,124,928.00",628.0,77.048432,81.033963,3318,4077,2719,66.680064,81.933280,54.642283,$585-630
Holden High School,Charter,427,"$248,087.00",581.0,83.803279,83.814988,395,411,381,92.505855,96.252927,89.227166,<$585


In [32]:
#  Calculate averages for the desired columns.
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()


/var/folders/pt/r0f84gpd53ndj2ptsk854sfw0000gn/T/ipykernel_19627/3377404583.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
/var/folders/pt/r0f84gpd53ndj2ptsk854sfw0000gn/T/ipykernel_19627/3377404583.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
/var/folders/pt/r0f84gpd53ndj2ptsk854sfw0000gn/T/ipykernel_19627/3377404583.py:4: FutureWarning: The default of observed=False

In [33]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({
    "Average Math Score": spending_math_scores,
    "Average Reading Score": spending_reading_scores,
    "% Passing Math": spending_passing_math,
    "% Passing Reading": spending_passing_reading,
    "% Overall Passing": overall_passing_spending
})

# Display results
spending_summary


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

In [34]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [35]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"],
                                                             size_bins, labels=labels,
                                                             include_lowest=True)

per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Students Passing Math and Reading,% Passing Math,% Passing Reading,% Overall Passing,School Size
School Name,,,,,,,,,,,,,
Huang High School,District,2917,"$1,910,635.00",655.0,76.629414,81.182722,1916,2372,1561,65.683922,81.316421,53.513884,Large (2000-5000)
Figueroa High School,District,2949,"$1,884,411.00",639.0,76.711767,81.158020,1946,2381,1569,65.988471,80.739234,53.204476,Large (2000-5000)
Shelton High School,Charter,1761,"$1,056,600.00",600.0,83.359455,83.725724,1653,1688,1583,93.867121,95.854628,89.892107,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",652.0,77.289752,80.934412,3094,3748,2481,66.752967,80.862999,53.527508,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",625.0,83.351499,83.816757,1371,1426,1330,93.392371,97.138965,90.599455,Medium (1000-2000)
Wilson High School,Charter,2283,"$1,319,574.00",578.0,83.274201,83.989488,2143,2204,2068,93.867718,96.539641,90.582567,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",582.0,83.061895,83.975780,1749,1803,1697,94.133477,97.039828,91.334769,Medium (1000-2000)
Bailey High School,District,4976,"$3,124,928.00",628.0,77.048432,81.033963,3318,4077,2719,66.680064,81.933280,54.642283,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",581.0,83.803279,83.814988,395,411,381,92.505855,96.252927,89.227166,Small (<1000)


In [36]:
# Calculate averages for the desired columns.
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

/var/folders/pt/r0f84gpd53ndj2ptsk854sfw0000gn/T/ipykernel_19627/2944522441.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
/var/folders/pt/r0f84gpd53ndj2ptsk854sfw0000gn/T/ipykernel_19627/2944522441.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
/var/folders/pt/r0f84gpd53ndj2ptsk854sfw0000gn/T/ipykernel_19627/2944522441.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True i

In [37]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame({
    "Average Math Score": size_math_scores,
    "Average Reading Score": size_reading_scores,
    "% Passing Math": size_passing_math,
    "% Passing Reading": size_passing_reading,
    "% Overall Passing": size_overall_passing
})

# Display results
size_summary


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [38]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [39]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({
    "Average Math Score": average_math_score_by_type,
    "Average Reading Score": average_reading_score_by_type,
    "% Passing Math": average_percent_passing_math_by_type,
    "% Passing Reading": average_percent_passing_reading_by_type,
    "% Overall Passing": average_percent_overall_passing_by_type})

# Display results
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
